<a href="https://colab.research.google.com/github/vegan111-vegan11/rag/blob/main/%EC%9D%B4%EC%A0%84_%EB%8C%80%ED%99%94%EB%A5%BC_%EA%B8%B0%EC%96%B5%ED%95%98%EB%8A%94_Langchain_%EC%82%AC%EC%9A%A9%EB%B2%95_4%EA%B0%80%EC%A7%80_%7C_%EB%81%9D%EB%A7%90%EC%9E%87%EA%B8%B0_%EC%B1%97%EB%B4%87_%EA%B5%AC%ED%98%84%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_openai
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.8/401.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.6/326.6 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existi

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk-..."

## 0. 환경설정

## 1. Langchain 제공 기능

참고 자료 : https://python.langchain.com/v0.2/docs/how_to/chatbots_memory/

### 1) Message passing
- 이전 대화 내용을 chain에 넘겨주는 것

- The simplest form of memory is simply passing chat history messages into a chain.

In [ ]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model="gpt-4o") # gpt-3.5-turbo-0125

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability. YOU MUST ANSWER IN KOREAN.",
        ),
        ("placeholder", "{messages}"),
    ]
)

chain = prompt | chat

ai_msg = chain.invoke(
    {
        "messages": [
            (
                "human",
                "일본어로 '잘 먹겠습니다.' 어떻게 말해? 한글 발음도 함께 말해줘.",
            ),
            ("ai",  "いただきます (이타다키마스)"),
            ("human", "내가 방금 뭐라고 했지?"),
        ],
    }
)
print(ai_msg.content)

당신은 일본어로 '잘 먹겠습니다.'라는 표현을 물어보셨습니다. 그리고 이에 대한 답으로 일본어로는 "いただきます" (이타다키마스)라고 말한다고 알려드렸습니다.


In [ ]:
ai_msg = chain.invoke(
    {
        "messages": [
            (
                "human",
                "내 이름은 공원나연이고, AI를 알려주는 유튜브를 운영하고 있어.",
            ),
            ("ai",  "그렇군요. 흥미로운데요?"),
            ("human", "내 이름이 뭐라고?"),
        ],
    }
)
print(ai_msg.content)

당신의 이름은 공원나연입니다.


- 응용 : 대화 쌓일 때마다 히스토리 저장하도록 구현

In [ ]:
history_list = []
while(True):
    user_input = input()
    if user_input == "종료": break
    history_list.append(
        (
            "human",
            user_input,
        )
    )
    print("## CHAT_HISTORY ##")
    print(history_list, "\n")
    ai_msg = chain.invoke(
        {
            "messages": history_list,
        }
    )
    print("AI Says : ",ai_msg.content)

    history_list.append(
        (
            "ai",
            ai_msg.content,
        )
    )

안녕
## CHAT_HISTORY ##
[('human', '안녕')] 

AI Says :  안녕하세요! 어떻게 도와드릴까요?
난 나연이라고 해.
## CHAT_HISTORY ##
[('human', '안녕'), ('ai', '안녕하세요! 어떻게 도와드릴까요?'), ('human', '난 나연이라고 해.')] 

AI Says :  안녕하세요, 나연님! 만나서 반갑습니다. 오늘 어떻게 도와드릴까요?
그리고 말차를 좋아하지
## CHAT_HISTORY ##
[('human', '안녕'), ('ai', '안녕하세요! 어떻게 도와드릴까요?'), ('human', '난 나연이라고 해.'), ('ai', '안녕하세요, 나연님! 만나서 반갑습니다. 오늘 어떻게 도와드릴까요?'), ('human', '그리고 말차를 좋아하지')] 

AI Says :  말차를 좋아하시는군요! 말차는 독특한 맛과 향이 있어서 많은 사람들이 좋아하죠. 말차를 자주 드시나요, 아니면 특별한 때에만 드시나요?
내 이름이 뭐였게?
## CHAT_HISTORY ##
[('human', '안녕'), ('ai', '안녕하세요! 어떻게 도와드릴까요?'), ('human', '난 나연이라고 해.'), ('ai', '안녕하세요, 나연님! 만나서 반갑습니다. 오늘 어떻게 도와드릴까요?'), ('human', '그리고 말차를 좋아하지'), ('ai', '말차를 좋아하시는군요! 말차는 독특한 맛과 향이 있어서 많은 사람들이 좋아하죠. 말차를 자주 드시나요, 아니면 특별한 때에만 드시나요?'), ('human', '내 이름이 뭐였게?')] 

AI Says :  나연님이시죠? 앞에서 말씀해 주셨습니다. 😊
굿
## CHAT_HISTORY ##
[('human', '안녕'), ('ai', '안녕하세요! 어떻게 도와드릴까요?'), ('human', '난 나연이라고 해.'), ('ai', '안녕하세요, 나연님! 만나서 반갑습니다. 오늘 어떻게 도와드릴까요?'), ('human', '그리고 말차를 좋아하지'), ('ai'

### 2) Chat history
- ChatMessageHistory() 클래스 사용하기
- It's perfectly fine to store and pass messages directly as an array, but we can use LangChain's built-in message history class to store and load messages as well.

https://api.python.langchain.com/en/latest/chat_message_histories/langchain_community.chat_message_histories.in_memory.ChatMessageHistory.html

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory

chat_history = ChatMessageHistory()

chat_history.add_user_message(
    "일본어로 '잘 먹겠습니다.' 어떻게 말해? 한글 발음도 함께 말해줘."
)

chat_history.add_ai_message("いただきます (이타다키마스)")

chat_history.messages
# chat_history.clear()

[HumanMessage(content="일본어로 '잘 먹겠습니다.' 어떻게 말해? 한글 발음도 함께 말해줘.", additional_kwargs={}, response_metadata={}),
 AIMessage(content='いただきます (이타다키마스)', additional_kwargs={}, response_metadata={})]

- 대화내용 바로바로 저장하기

In [ ]:
chat_history = ChatMessageHistory()

################ USER INPUT - 1 ################
input1 = "일본어로 '잘 먹겠습니다.' 어떻게 말해? 한글 발음도 함께 말해줘."

# 대화 히스토리에 입력(user_input) 저장
chat_history.add_user_message(input1)
print("첫번째 입력 후 history : ", chat_history.messages)

################ AI RESPONSE - 1 ################
# 답변 생성(response)
response = chain.invoke(
    {
        "messages": chat_history.messages,
    }
)

# 대화 히스토리에 답변(response) 저장
chat_history.add_ai_message(response)
print("첫번째 답변 후 history : ", chat_history.messages)
print()

################ USER INPUT - 2 ################
input2 = "내가 방금 뭐라고 물어봤지?"

# 대화 히스토리에 입력(user_input) 저장
chat_history.add_user_message(input2)
print("두번째 입력 후 history : ", chat_history.messages)

################ AI RESPONSE - 2 ################
# 답변 생성(response)
print("[두번째 입력에 대한 답변]")
chain.invoke(
    {
        "messages": chat_history.messages,
    }
)

첫번째 입력 후 history :  [HumanMessage(content="일본어로 '잘 먹겠습니다.' 어떻게 말해? 한글 발음도 함께 말해줘.", additional_kwargs={}, response_metadata={})]
첫번째 답변 후 history :  [HumanMessage(content="일본어로 '잘 먹겠습니다.' 어떻게 말해? 한글 발음도 함께 말해줘.", additional_kwargs={}, response_metadata={}), AIMessage(content='일본어로 \'잘 먹겠습니다.\'는 "いただきます"라고 말합니다. 한글 발음은 "이타다키마스"입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 59, 'total_tokens': 93, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_2f406b9113', 'finish_reason': 'stop', 'logprobs': None}, id='run-1ed0525d-6e28-4e28-9c7a-a8970c5b9c49-0', usage_metadata={'input_tokens': 59, 'output_tokens': 34, 'total_tokens': 93, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})]

두번째 입력 후 history :  [HumanMessage(content="일본어로 '잘

AIMessage(content='당신은 "일본어로 \'잘 먹겠습니다.\' 어떻게 말해? 한글 발음도 함께 말해줘."라고 물어보셨습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 112, 'total_tokens': 146, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_e5e4913e83', 'finish_reason': 'stop', 'logprobs': None}, id='run-8a35d5c0-d930-4eef-a5fa-11cf78860163-0', usage_metadata={'input_tokens': 112, 'output_tokens': 34, 'total_tokens': 146, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

- 응용 : 대화 쌓일 때마다 히스토리 저장하도록 구현

In [ ]:
chat_history = ChatMessageHistory()

while(True):
    user_input = input()
    if user_input == "종료": break
    chat_history.add_user_message(user_input)
    response = chain.invoke(
        {
            "messages": chat_history.messages,
        }
    )
    chat_history.add_ai_message(response)
    print(response.content)

안녕!
안녕하세요! 무엇을 도와드릴까요?
말차로 만든 디저트에는 뭐가 있어?
말차로 만든 디저트에는 다양한 종류가 있습니다. 몇 가지 예를 들면:

1. **말차 티라미수** - 전통적인 티라미수에 말차를 더해 새로운 맛을 즐길 수 있습니다.
2. **말차 마카롱** - 겉은 바삭하고 속은 부드러운 마카롱에 말차 크림을 채워 넣은 디저트입니다.
3. **말차 아이스크림** - 부드럽고 진한 말차 맛이 느껴지는 아이스크림입니다.
4. **말차 롤케이크** - 부드러운 스폰지 케이크에 말차 크림을 발라 말아서 만든 케이크입니다.
5. **말차 브라우니** - 진한 초콜릿 브라우니에 말차를 더해 새로운 풍미를 더한 디저트입니다.
6. **말차 푸딩** - 부드럽고 크리미한 말차 맛의 푸딩입니다.
7. **말차 파운드 케이크** - 촉촉하고 진한 말차 맛이 느껴지는 파운드 케이크입니다.

이 외에도 다양한 창의적인 말차 디저트들이 있으니 시도해보시면 좋을 것 같습니다!
이 중에서 가장 만들기 쉬운건 뭘까?
말차 디저트 중에서 비교적 만들기 쉬운 것은 **말차 아이스크림**이나 **말차 푸딩**입니다. 

- **말차 아이스크림**: 아이스크림 제조기에 필요한 재료를 섞어서 얼리기만 하면 되기 때문에 비교적 간단합니다. 만약 아이스크림 제조기가 없다면, 생크림과 연유, 말차 가루를 섞어 냉동실에 얼리면서 중간중간 젓는 방법으로도 만들 수 있습니다.

- **말차 푸딩**: 젤라틴을 사용하여 푸딩을 굳히는 방식이라 복잡한 베이킹 기술이 필요하지 않습니다. 우유, 설탕, 말차 가루, 젤라틴을 잘 섞어서 굳히기만 하면 되므로 초보자도 쉽게 만들 수 있습니다.

이 두 가지는 비교적 간단한 재료와 과정을 통해 만들 수 있으니 처음 시도해보기 좋습니다.
그럼 말차 브라우니는?
말차 브라우니는 기본적인 베이킹 기술이 필요하지만, 비교적 간단하게 만들 수 있는 디저트 중 하나입니다. 여기 간단한 말차 브라우니 레시피를 소개해드릴게요.

**재료:**
- 밀가루 100g
- 설탕 1

### 3) Automatic history management
- RunnableWithMessageHistory() 활용하여 메모리 관리하기
- The previous examples pass messages to the chain explicitly. This is a completely acceptable approach, but it does require external management of new messages. LangChain also includes an wrapper for LCEL chains that can handle this process automatically called RunnableWithMessageHistory.

In [ ]:
chat = ChatOpenAI(model="gpt-4o")
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability. YOU MUST ANSWER IN KOREAN.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

chain = prompt | chat

In [ ]:
from langchain_core.runnables.history import RunnableWithMessageHistory

chat_history_for_chain = ChatMessageHistory()

chain_with_message_history = RunnableWithMessageHistory(
    chain, # 실행할 Runnable 객체
    lambda session_id: chat_history_for_chain, # 세션 기록을 가져오는 함수
    input_messages_key="input", # 입력 메시지의 Key
    history_messages_key="chat_history", # 대화 히스토리 메시지의 Key
)

In [ ]:
chain_with_message_history.invoke(
    {"input": "일본어로 '잘 먹겠습니다.' 어떻게 말해? 한글 발음도 함께 말해줘."},
    {"configurable": {"session_id": "unused"}},
)

AIMessage(content='일본어로 \'잘 먹겠습니다.\'는 "いただきます"라고 말합니다. 한글 발음은 "이타다키마스"입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 59, 'total_tokens': 93, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_e5e4913e83', 'finish_reason': 'stop', 'logprobs': None}, id='run-6ea25a7f-eaa5-4809-81eb-57d41ad37427-0', usage_metadata={'input_tokens': 59, 'output_tokens': 34, 'total_tokens': 93, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [ ]:
chain_with_message_history.invoke(
    {"input": "내가 뭐라고 물어봤지?"}, {"configurable": {"session_id": "unused"}}
)

AIMessage(content='당신은 "\'잘 먹겠습니다.\'를 일본어로 어떻게 말하는지" 물어보셨습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 110, 'total_tokens': 133, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_e5e4913e83', 'finish_reason': 'stop', 'logprobs': None}, id='run-50037728-4fa6-4506-ba54-a5d97a92e832-0', usage_metadata={'input_tokens': 110, 'output_tokens': 23, 'total_tokens': 133, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

히스토리 저장 함수를 따로 호출할 필요없이 invoke 실행만으로 대화 기록이 자동 저장되는 것을 확인 가능!

### 4) Modifying chat history
- 이전 대화 내용을 가공하여 넘겨주기
- Modifying stored chat messages can help your chatbot handle a variety of situations.

In [ ]:
chat_history = ChatMessageHistory()

chat_history.add_user_message("안녕하세요. 제 이름은 나연입니다.")
chat_history.add_ai_message("안녕하세요, 나연님! 무엇을 도와드릴까요?")
chat_history.add_user_message("날씨 좋은 날 들을만 한 노래 추천해주세요.")
chat_history.add_ai_message("Carpenters - Close to you 를 추천해요.")

chat_history.messages
# chat_history.clear()

[HumanMessage(content='안녕하세요. 제 이름은 나연입니다.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요, 나연님! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='날씨 좋은 날 들을만 한 노래 추천해주세요.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Carpenters - Close to you 를 추천해요.', additional_kwargs={}, response_metadata={})]

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability. The provided chat history includes facts about the user you are speaking with. YOU MUST ANSWER IN KOREAN.",
        ),
        ("placeholder", "{chat_history}"),
        ("user", "{input}"),
    ]
)

chain = prompt | chat

chain_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [ ]:
from langchain_core.runnables import RunnablePassthrough

def summarize_messages(chain_input):
    stored_messages = chat_history.messages
    if len(stored_messages) == 0:
        return False
    summarization_prompt = ChatPromptTemplate.from_messages(
        [
            ("placeholder", "{chat_history}"),
            (
                "user",
                "Distill the above chat messages into a single summary message. Include as many specific details as you can.",
            ),
        ]
    )
    summarization_chain = summarization_prompt | chat

    # chat_history 에 저장된 대화 기록을 요약프롬프트에 입력 & 결과 저장
    summary_message = summarization_chain.invoke({"chat_history": stored_messages})

    # chat_history 에 저장되어있던 기록 지우기
    chat_history.clear()

    # 생성된 새로운 요약내용으로 기록 채우기
    chat_history.add_message(summary_message)

    return True

chain_with_summarization = (
    RunnablePassthrough.assign(messages_summarized=summarize_messages)
    | chain_with_message_history
)

- summarize_messages 함수의 실행 결과를 messages_summarized 라는 이름의 Key 로 저장

    - 정의한 chain 안에 "messages_summarized" Key 가 정의되고 있지 않으므로, 이는 `summarize_messages` 함수를 실행하기 위한 무시 가능한 Key
    - 함수가 실행되면 chat_history 에 요약된 히스토리가 저장됨


In [ ]:
chain_with_summarization.invoke(
    {"input": "제 이름을 기억하고 있나요?"},
    {"configurable": {"session_id": "unused"}},
)

AIMessage(content='네, 나연님이라고 알고 있습니다. 맞나요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 94, 'total_tokens': 106, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_2f406b9113', 'finish_reason': 'stop', 'logprobs': None}, id='run-0e55a4c3-cc75-4b51-afc6-41fc57f30d60-0', usage_metadata={'input_tokens': 94, 'output_tokens': 12, 'total_tokens': 106, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [ ]:
chain_with_summarization.invoke(
    {"input": "그 가수는 남자인가요 여자인가요?"},
    {"configurable": {"session_id": "unused"}},
)

AIMessage(content='Carpenters는 남매 듀오로, 리처드 카펜터와 카렌 카펜터로 구성되어 있습니다. 카렌 카펜터는 여성 보컬리스트입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 109, 'total_tokens': 153, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_e5e4913e83', 'finish_reason': 'stop', 'logprobs': None}, id='run-20fa421b-37ee-49f2-8af2-dbc6ea408fd6-0', usage_metadata={'input_tokens': 109, 'output_tokens': 44, 'total_tokens': 153, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

## 2. 끝말잇기 게임 구현하기

- 4) Modifying chat history 의 활용!

In [ ]:
# 대화 기록을 저장할 히스토리 클래스 불러오기
chat_history = ChatMessageHistory()

# chat_history.add_user_message("끝말잇기 하자")
# chat_history.add_ai_message("좋습니다. 제가 먼저 시작할게요. 바나나!")
# chat_history.add_user_message("나이테")
# chat_history.add_ai_message("테이프")

chat_history.messages

[]

In [ ]:
chat = ChatOpenAI(model="gpt-4o")
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """당신은 끝말잇기 게임을 진행하는 AI 챗봇입니다. 아래는 게임 규칙입니다. 당신과 user 의 입력에서 아래 규칙이 꼭 지켜져야 하며, 지키지 않은 사람에게 패배를 알린 뒤, 끝말잇기 게임을 종료합니다.
                1. 주어진 대화 기록에서 이미 나왔던 단어를 다시 말했을 경우 패배합니다.
                2. 두음법칙을 허용합니다. (ex. 리 -> 이, 력 -> 역, 락 -> 낙)
                3. 국어사전에 존재하는 단어이자, 명사여야 합니다.
            """,
        ),
        ("placeholder", "{chat_history}"),
        ("user", "{input}"),
    ]
)

chain = prompt | chat

chain_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [ ]:
def summarize_messages(chain_input):
    stored_messages = chat_history.messages
    if len(stored_messages) == 0:
        return False
    summarization_prompt = ChatPromptTemplate.from_messages(
        [
            ("placeholder", "{chat_history}"),
            (
                "user",
                "위 채팅 메시지는 끝말잇기 게임을 진행한 대화내용입니다. 언급한 단어들만 나열하여 저장해주세요.",
            ),
        ]
    )
    summarization_chain = summarization_prompt | chat

    # chat_history 에 저장된 대화 기록을 요약프롬프트에 입력 & 결과 저장
    summary_message = summarization_chain.invoke({"chat_history": stored_messages})

    # chat_history 에 저장되어있던 기록 지우기
    chat_history.clear()

    # 생성된 새로운 요약내용으로 기록 채우기
    chat_history.add_message(summary_message)

    return True


chain_with_summarization = (
    RunnablePassthrough.assign(messages_summarized=summarize_messages)
    | chain_with_message_history
)

In [ ]:
while(True):
    user_input = input("🙋‍♂️ YOUR TURN : ")
    if user_input == "종료": break
    response = chain_with_summarization.invoke(
                {"input": user_input},
                {"configurable": {"session_id": "unused"}},
            )
    print("✍ AI TURN : ", response.content)

🙋‍♂️ YOUR TURN : 바나나
✍ AI TURN :  나무
🙋‍♂️ YOUR TURN : 무지개
✍ AI TURN :  개구리
🙋‍♂️ YOUR TURN : 이빨
✍ AI TURN :  빨래
🙋‍♂️ YOUR TURN : 래미콘
✍ AI TURN :  콘서트
🙋‍♂️ YOUR TURN : 트럭
✍ AI TURN :  럭비
🙋‍♂️ YOUR TURN : 비행기
✍ AI TURN :  기차
🙋‍♂️ YOUR TURN : 차표
✍ AI TURN :  표범
🙋‍♂️ YOUR TURN : 범고래
✍ AI TURN :  래프팅
🙋‍♂️ YOUR TURN : 팅팅탱탱
✍ AI TURN :  죄송하지만 "팅팅탱탱"은 국어사전에 존재하는 명사가 아닙니다. 따라서 규칙을 어기셨습니다. 당신의 패배로 끝말잇기 게임을 종료합니다.
🙋‍♂️ YOUR TURN : 종료
